Attempt to solve the problem in 2 stages. First, try to determine if this is a whale we've seen before. Secondly for each whale we have seen before, match the tail to an id.

# Pre-trained model

In [1]:
from tensorflow import keras
import os

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [2]:
base_model.trainable = False

# Add new layers

In [3]:
import numpy as np
import pandas as pd

train_map_df = pd.read_csv(os.path.join("data", "train.csv"))
train_map_df['Id'] = train_map_df['Id'].astype('category')
train_map_df['is_new'] = np.where(train_map_df['Id'] == 'new_whale', 'new', 'existing')
train_map_df.head()

,Image,Id,is_new
0,0000e88ab.jpg,w_f48451c,existing
1,0001f9222.jpg,w_c3d896a,existing
2,00029d126.jpg,w_20df2c5,existing
3,00050a15a.jpg,new_whale,new
4,0005c1ef8.jpg,new_whale,new


In [4]:
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1)(x) # A Dense classifier with a class for new/existing
model = keras.Model(inputs, outputs)

In [5]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________


# Compiling Model

In [6]:
# we have a binary problem (either new or not new)
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])

# Augment the Data

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,
        vertical_flip=False,
        validation_split=0.2)  # 80/20 split train/test

In [8]:
train_it = datagen.flow_from_dataframe(train_map_df,
                                       directory=os.path.join('data', 'train'), 
                                       x_col='Image',
                                       y_col='is_new',
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='binary', 
                                       batch_size=8,
                                       subset='training')
valid_it = datagen.flow_from_dataframe(train_map_df,
                                       directory=os.path.join('data', 'train'), 
                                       x_col='Image',
                                       y_col='is_new',
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='binary', 
                                       batch_size=8,
                                       subset='validation')

Found 20289 validated image filenames belonging to 2 classes.
Found 5072 validated image filenames belonging to 2 classes.


In [9]:
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=20)

Epoch 1/20
12/12 [==============================] - 12s 1s/step - loss: 1.2056 - binary_accuracy: 0.6354 - val_loss: 1.2087 - val_binary_accuracy: 0.5312
Epoch 2/20
12/12 [==============================] - 17s 1s/step - loss: 1.1791 - binary_accuracy: 0.6146 - val_loss: 0.9781 - val_binary_accuracy: 0.6875
Epoch 3/20
12/12 [==============================] - 19s 2s/step - loss: 1.4556 - binary_accuracy: 0.5417 - val_loss: 1.2613 - val_binary_accuracy: 0.5312
Epoch 4/20
12/12 [==============================] - 20s 2s/step - loss: 1.1968 - binary_accuracy: 0.5938 - val_loss: 1.2526 - val_binary_accuracy: 0.6250
Epoch 5/20
12/12 [==============================] - 19s 2s/step - loss: 1.4579 - binary_accuracy: 0.6042 - val_loss: 0.9270 - val_binary_accuracy: 0.6250
Epoch 6/20
12/12 [==============================] - 19s 2s/step - loss: 1.0840 - binary_accuracy: 0.6250 - val_loss: 0.5057 - val_binary_accuracy: 0.8125
Epoch 7/20
12/12 [==============================] - 14s 1s/step - loss: 0.95

In [10]:
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input

predictions = []

def make_prediction(image_path):
    image = image_utils.load_img(image_path, target_size=(224, 224))
    image = image_utils.img_to_array(image)
    image = image.reshape(1,224,224,3)
    image = preprocess_input(image)
    preds = model.predict(image)
    return preds

for id in train_map_df['Image']:
    predictions.append(0 if make_prediction(os.path.join('data', 'train', id))[0][0] < 0 else 1)

train_map_df['predicted_new'] = pd.Series(predictions)
train_map_df

,Image,Id,is_new,predicted_new
0,0000e88ab.jpg,w_f48451c,existing,0
1,0001f9222.jpg,w_c3d896a,existing,0
2,00029d126.jpg,w_20df2c5,existing,0
3,00050a15a.jpg,new_whale,new,1
4,0005c1ef8.jpg,new_whale,new,0
...,...,...,...,...
25356,ffef89eed.jpg,w_9c506f6,existing,0
25357,fff7faf61.jpg,w_9cf0388,existing,0
25358,fff9002e0.jpg,w_bd1c3d5,existing,0
25359,fffcde6fe.jpg,w_9f30885,existing,0


Secondary model to id existing whales.

In [12]:
base_model_2 = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

base_model_2.trainable = False

inputs_2 = keras.Input(shape=(224, 224, 3))
x_2 = base_model(inputs_2, training=False)
x_2 = keras.layers.GlobalAveragePooling2D()(x_2)
outputs_2 = keras.layers.Dense(len(train_map_df['Id'].unique())-1)(x_2) # A Dense classifier with a each class from our training set excluded new_whale
model_2 = keras.Model(inputs_2, outputs_2)

# we have a categorical problem so need to use CategoricalCrossentropy and CategoricalAccuracy
model_2.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True), metrics=[keras.metrics.CategoricalAccuracy()])

datagen_2 = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,
        vertical_flip=False,
        validation_split=0.2)  # 80/20 split train/test

train_it_2 = datagen.flow_from_dataframe(train_map_df[train_map_df['is_new'] == 'existing'],
                                       directory=os.path.join('data', 'train'), 
                                       x_col='Image',
                                       y_col='Id',
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='categorical', 
                                       batch_size=8,
                                       subset='training')
valid_it_2 = datagen.flow_from_dataframe(train_map_df[train_map_df['is_new'] == 'existing'],
                                       directory=os.path.join('data', 'train'), 
                                       x_col='Image',
                                       y_col='Id',
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='categorical', 
                                       batch_size=8,
                                       subset='validation')
model_2.fit(train_it_2, steps_per_epoch=12, validation_data=valid_it_2, validation_steps=4, epochs=20)

Found 12558 validated image filenames belonging to 5004 classes.
Found 3139 validated image filenames belonging to 5004 classes.
Epoch 1/20
12/12 [==============================] - 14s 1s/step - loss: 11.0963 - categorical_accuracy: 0.0000e+00 - val_loss: 11.5492 - val_categorical_accuracy: 0.0000e+00
Epoch 2/20
12/12 [==============================] - 17s 1s/step - loss: 11.7758 - categorical_accuracy: 0.0000e+00 - val_loss: 11.4979 - val_categorical_accuracy: 0.0000e+00
Epoch 3/20
12/12 [==============================] - 20s 2s/step - loss: 11.4828 - categorical_accuracy: 0.0000e+00 - val_loss: 12.4477 - val_categorical_accuracy: 0.0000e+00
Epoch 4/20
12/12 [==============================] - 19s 2s/step - loss: 12.4159 - categorical_accuracy: 0.0000e+00 - val_loss: 12.2240 - val_categorical_accuracy: 0.0000e+00
Epoch 5/20
12/12 [==============================] - 14s 1s/step - loss: 12.2706 - categorical_accuracy: 0.0104 - val_loss: 12.7259 - val_categorical_accuracy: 0.0000e+00
Epoch